In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [3]:
%cd drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.layers import Input, Dense, Embedding, LSTM
from keras.models import Model
import keras
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil
import globalobject as g
from tqdm import tqdm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#f = open(g.text_file + ".txt", 'r')
f = open('/content/drive/MyDrive/Colab Notebooks/dataset/' + g.text_file + '.txt', 'r')

In [7]:
load_address = list()

f.seek(0, 0)
while True:
  line = f.readline()
  if not line: break
  split_line = line.split(', ')
  load_address.append(split_line[2])

for i in range(len(load_address)):
  load_address[i] = int(load_address[i], 16)//64

print("\nload_address example: ")
print(load_address[:100])


load_address example: 
[702778646977, 702778647164, 702778646976, 702778646921, 702778647176, 702778647287, 702778647353, 702778647368, 702778647452, 702778647286, 702778647484, 702778647560, 702778647644, 702778647688, 702778647352, 702778647755, 702778647819, 702778647901, 702778647933, 4363500793931, 702778647351, 702778647820, 702778647756, 4363500793932, 4363500794023, 4363500794087, 174026416743, 702778647350, 174026416807, 174026416838, 2627809326406, 2627809326492, 2627809326524, 2627809326534, 2627809326620, 702778647349, 2627809326662, 2627809326780, 2627809326844, 2627809326857, 2627809326889, 2627809326972, 2627809326988, 2627809327052, 2627809327116, 2627809327180, 2627809327277, 2627809327309, 2627809327341, 702778647348, 2627809327115, 2627809327366, 2627809327430, 2627809327532, 2627809327564, 2627809327596, 2627809327622, 2627809327686, 2627809327749, 702778647347, 2627809327813, 2627809327877, 2627809327979, 2627809328011, 2627809328043, 2627809327947, 2627809328069,

In [8]:
delta = list()
for i in range(len(load_address)-1):
  delta.append(load_address[i+1] - load_address[i])

for i in range(len(delta)):
  if delta[i] > 500:
    delta[i] = 1000
    continue
  if delta[i] < -500:
    delta[i] = 0
    continue
  delta[i] += 500

print("\ndelta example, final delta:")
print(delta[0:100])
print(delta[-1])

delta_bundle = list()
for i in range(len(delta) - g.seq_length):
  delta_bundle.append(delta[i:i+g.seq_length+1])
delta_bundle = np.array(delta_bundle)

print("\ndelta_bundle example:")
print(delta_bundle)

rnn_data = delta_bundle[:,:-1]
rnn_data = np.array(rnn_data)
print("\nfirst input example(rnn_data): ")
print(rnn_data)


delta example, final delta:
[687, 312, 445, 755, 611, 566, 515, 584, 334, 698, 576, 584, 544, 164, 903, 564, 582, 532, 1000, 0, 969, 436, 1000, 591, 564, 0, 1000, 0, 531, 1000, 586, 532, 510, 586, 0, 1000, 618, 564, 513, 532, 583, 516, 564, 564, 564, 597, 532, 532, 0, 1000, 751, 564, 602, 532, 532, 526, 564, 563, 0, 1000, 564, 602, 532, 532, 404, 622, 564, 0, 1000, 564, 597, 531, 437, 370, 768, 564, 589, 532, 571, 461, 603, 301, 763, 589, 0, 1000, 0, 1000, 1000, 0, 532, 347, 685, 1000, 0, 781, 0, 1000, 563, 501]
585

delta_bundle example:
[[ 687  312  445  755  611  566]
 [ 312  445  755  611  566  515]
 [ 445  755  611  566  515  584]
 ...
 [ 499    0  554  501 1000  499]
 [   0  554  501 1000  499    0]
 [ 554  501 1000  499    0  585]]

first input example(rnn_data): 
[[ 687  312  445  755  611]
 [ 312  445  755  611  566]
 [ 445  755  611  566  515]
 ...
 [ 499    0  554  501 1000]
 [   0  554  501 1000  499]
 [ 554  501 1000  499    0]]


In [9]:
# 8 -> [0,0,0,1] change int to categorical value
def address_to_binary(a):
  binary_int = list()
  for i in range(g.MAXLEN):
    # /1000 -> scaling
    binary_int.append(a%2)
    a = a//2
  return binary_int

address_binary_input = list()
for i in range(g.seq_length, len(load_address)-1):
  address_binary_input.append(address_to_binary(load_address[i]))

print("\naddress binary input example")
print(address_binary_input[0])
address_binary_input = np.array(address_binary_input)


address binary input example
[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0]


In [10]:
delta_output = delta_bundle[:,-1:]
delta_output = delta_output.flatten()
print("\noutput example:")
print(delta_output)

print('output shape')
print(rnn_data.shape)
print(address_binary_input.shape)
print(delta_output.shape)


output example:
[566 515 584 ... 499   0 585]
output shape
(721566, 5)
(721566, 43)
(721566,)


In [11]:
delete_list = list()
for i in range(len(delta_output)):
  if delta_output[i] == 0 or delta_output[i] == 1000:
    delete_list.append(i)

delta_output = np.delete(delta_output, delete_list)
rnn_data = np.delete(rnn_data, delete_list, 0)
address_binary_input = np.delete(address_binary_input, delete_list, 0)

print("\n0 and 1000 removed delta output example:")
print(delta_output)


0 and 1000 removed delta output example:
[566 515 584 ... 501 499 585]


In [12]:
idx = np.arange(rnn_data.shape[0])
np.random.shuffle(idx)

rnn_data = rnn_data[idx]
address_binary_input = address_binary_input[idx]
delta_output = delta_output[idx]

cut_index = len(rnn_data) * g.train_model_percentage // 100

train_rnn_data = rnn_data[:cut_index,:]
test_rnn_data = rnn_data[cut_index:]

train_address_binary_input = address_binary_input[:cut_index,:]
test_address_binary_input = address_binary_input[cut_index:,:]

train_delta_output = delta_output[:cut_index]
test_delta_output = delta_output[cut_index:]

In [13]:
np.save('/content/drive/MyDrive/Colab Notebooks/dataset/' + g.text_file + '_train_rnn_data', train_rnn_data)
np.save('/content/drive/MyDrive/Colab Notebooks/dataset/' + g.text_file + '_test_rnn_data', test_rnn_data)
np.save('/content/drive/MyDrive/Colab Notebooks/dataset/' + g.text_file + '_train_address_binary_input', train_address_binary_input)
np.save('/content/drive/MyDrive/Colab Notebooks/dataset/' + g.text_file + '_test_address_binary_input', test_address_binary_input)
np.save('/content/drive/MyDrive/Colab Notebooks/dataset/' + g.text_file + '_train_delta_output', train_delta_output)
np.save('/content/drive/MyDrive/Colab Notebooks/dataset/' + g.text_file + '_test_delta_output', test_delta_output)